In [7]:
import os
import json
import re
from groq import Groq

client = Groq(
    api_key="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",
)

# Modify the prompt to request JSON output
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": (
                "Explain the importance of fast language models. "
                "Provide the answer in JSON format with keys 'summary', 'benefits', and 'applications'."
            ),
        }
    ],
    model="llama3-8b-8192",
)

content = chat_completion.choices[0].message.content

# Use regex to extract JSON content in case there is extra text
json_match = re.search(r'\{.*\}', content, re.DOTALL)

if json_match:
    json_content = json_match.group()
    try:
        data = json.loads(json_content)
        print(json.dumps(data, indent=4))
    except json.JSONDecodeError as e:
        print("Failed to parse JSON:", e)
        print("Content was:", json_content)
else:
    print("No JSON content found in the response.")


{
    "summary": {
        "text": "Fast language models have revolutionized the field of natural language processing (NLP) by enabling efficient and accurate processing of vast amounts of text data. They have numerous applications in various industries and have become a crucial component of many AI-powered systems.",
        "importance": "Fast language models are important because they can process and generate human-like language at high speeds, making them useful for tasks that require quick responses or decisions."
    },
    "benefits": [
        {
            "name": "Improved Accuracy",
            "description": "Fast language models are trained on large datasets and can generate more accurate results than traditional language models."
        },
        {
            "name": "Faster Processing",
            "description": "They can process large amounts of text data quickly, making them suitable for real-time applications."
        },
        {
            "name": "Enhanced Us

In [ ]:
#LLM Inference Default

import requests

api_key = "ljoWc7l91mdURBRNTNPlZQlGpV5OMo"  # Replace with your actual API key

url = "http://localhost:8000/chat/completions"
headers = {
    "Content-Type": "application/json",
    "api-key": api_key,
}

data = {
    "messages": [
        {"role": "user", "content": "Explain the importance of fast language models"}
    ],
    "model": "llama3.1-8b-8192",
    "service_name": "cerebras",
}

response = requests.post(url, json=data, headers=headers)

if response.status_code == 200:
    result = response.json()
    print(result["content"])
else:
    print(f"Error {response.status_code}: {response.text}")

In [ ]:
#LLM Inference Custom Parameters

import requests

api_key = "ljoWc7l91mdURBRNTNPlZQlGpV5OMo"  # Replace with your actual API key

url = "http://localhost:8000/chat/completions"
headers = {
    "Content-Type": "application/json",
    "api-key": api_key,
}

data = {
  "messages": [
        {"role": "user", "content": "Explain the importance of fast language models"}
    ],
  "model": "llama3.1-8b",
  "service_name": "cerebras",
  "temperature": 0.7,
  "max_tokens": 500,
  "top_p": 0.9,
  "stop": ["\n"]
}

response = requests.post(url, json=data, headers=headers)

if response.status_code == 200:
    result = response.json()
    print(result["content"])
else:
    print(f"Error {response.status_code}: {response.text}")

In [11]:
import requests

api_key = "ljoWc7l91mdURBRNTNPlZQlGpV5OMo"  # Replace with your actual API key
url = "http://localhost:8000/chat/completions"  # Adjust the URL if necessary

# System prompt
system_prompt = """
You're evaluating writing style in text.
Your evaluations must always be in JSON format. Here is an example JSON response:

{ "readability": 4, "conciseness": 2 }

"""

# User prompt
tweet = "This is a sample text that needs evaluation."
prompt = f"""
Evaluate the text:

{tweet}

You're evaluating the readability and conciseness with values from 0 (extremely bad) to 10 (extremely good).
"""

# Messages
messages = [    {"role": "system", "content": system_prompt},    {"role": "user", "content": prompt}]

# Request data
data = {
    "messages": messages,
    "model": "llama3.1-8b",  # Replace with your desired model
    "service_name": "cerebras",     # Replace with your desired service
    "temperature": 0.0,         # Low temperature for deterministic output
    "max_tokens": 100,          # Adjust as needed
    "top_p": 1.0,
}

headers = {
    "Content-Type": "application/json",
    "api-key": api_key,
}

# Make the API call
response = requests.post(url, json=data, headers=headers)

if response.status_code == 200:
    result = response.json()
    print(result["content"])
else:
    print(f"Error {response.status_code}: {response.text}")

{
  "readability": 8,
  "conciseness": 6
}


In [ ]:

model = ChatOllama(model="mistral:7b-instruct-v0.2-q4_1")
parser = StrOutputParser()
output = 'News_data_Company_SP500_3_Sentiment_Score.xlsx'
Question = """
    In this format respond to the news aricle given in the context interpret it and classify and respond in this JSON format only. No other things and extra explaination or interpretation.
    JSON = {
        Sentiment: Positive/Negative/Neutral,
        Corruption: Yes/No/Possible,
        Fraud: Yes/No/Possible,
        Type: (Type of news article),
        Explanation: (Write Explanation Here),
    }
    Return just the JSON No matter what. Strictly the formatted JSON.
"""

In [ ]:
def is_valid_json(json_data):
    try:
        json_object = json.loads(json_data)
    except ValueError as e:
        return False
    return True

def fix_json(json_data):
    try:
        if not json_data.strip().endswith('}'):
            json_data += '}'
        return json_data
    except Exception as e:
        return json_data
    
def perform_sentiment_analysis(text, model, parser, Question):
    template = """
    This is a news article from Thompson Reuters. Use this news article and respond only in the format asked in the question nothing extra. Do not respond with anything else just the JSON format.

    Context: {context}

    Question: {question}
    """

    prompt = ChatPromptTemplate.from_template(template)
    chain = prompt | model | parser
    response = chain.invoke({
        "context": text,
        "question": Question
    })
    if not is_valid_json(response):
        fixed_response = fix_json(response)
        if is_valid_json(fixed_response):
            return json.loads(fixed_response)
        else:
            response = {
                'Sentiment': None,
                'Corruption': None,
                'Fraud': None,
                'Type': None,
                'Explanation': "Error in Analysis"
            }
            return(response)
    else:
        return json.loads(response)